###### References: 
- https://docs.python.org/3/library/functions.html   
- Fluent Python by Luciano Ramalho. Chapter 5: First-Class Functions

# First-Class Functions
- Created at runtime
- Assigned to a variable or element in a data structure
- Passed as an argument to a function
- Returned as the result of a funtion

# Treating a Function Like an Object

In [1]:
def factorial(n):
    '''returns n!'''
    return 1 if n < 2 else n* factorial(n-1)

In [2]:
factorial(42)

1405006117752879898543142606244511569936384000000000

In [3]:
factorial.__doc__

'returns n!'

In [4]:
type(factorial)

function

Use function through a different name and pass as function:

In [5]:
fact = factorial

In [6]:
fact

<function __main__.factorial(n)>

In [7]:
fact(5)

120

In [8]:
map(factorial, range(11))

In [9]:
list(map(fact, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

# Higher-Order Functions
A function that takes a function as argument or returns a function as the result.

In [10]:
fruits = ['papaya', 'mangosteen', 'durian', 'ciku', 'duku langsat', 'guava', 'coconut']
sorted(fruits, key=len)

['ciku', 'guava', 'papaya', 'durian', 'coconut', 'mangosteen', 'duku langsat']

In [11]:
def reverse(word):
    return word[::-1]
reverse('testing')

'gnitset'

In [12]:
sorted(fruits, key=reverse)

['guava', 'papaya', 'durian', 'mangosteen', 'duku langsat', 'coconut', 'ciku']

## Modern replacement for `map`, `filter` and `reduce`
replaced by listcomp or genexp

In [13]:
list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [14]:
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [15]:
list(map(factorial, filter(lambda n: n % 2, range(6))))

[1, 6, 120]

In [16]:
[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

Sum of integers up to 99 

In [17]:
from functools import reduce
from operator import add

In [18]:
reduce(add, range(100))

4950

In [19]:
sum(range(100))

4950

### `all(iterable)`
### `any(iterable)`
# Anonymous Functions
`lambda`

In [20]:
sorted(fruits, key=lambda word: word[::-1])

['guava', 'papaya', 'durian', 'mangosteen', 'duku langsat', 'coconut', 'ciku']

# The Seven Flavours of Callable Objects
call operator: `()`
 - User defined functions
 - Built-in functions
 - Built-in methods
 - Methods
 - Classes
 - Class intances
 - Generator functions

In [21]:
abs, str, 13

(<function abs(x, /)>, str, 13)

In [22]:
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

## User Defined Callable Types
### `__call__()`
picks item from a shuffled list

In [23]:
import random

In [24]:
class BingoCage:
    def __init__(self, items):
        self._items = list(items) # build a local copy of any iterable
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self): # shortcut to bingo.pick()
        return self.pick()

In [25]:
bingo = BingoCage(range(3))
bingo.pick()

2

In [26]:
bingo()

1

In [27]:
callable(bingo)

True

# Function Introspection

In [28]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

Listing attributes of functions that don't exist in plain instances

In [29]:
class C: pass    # create bare user defined class 
obj = C()
def func(): pass # create a bare function

In [30]:
sorted(set(dir(func)) - set(dir(obj))) # generate a sorted list of attributes that exist in a function but not in class

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

# From Positional to Keyword-Only Parameters
a keyword-only argument cls is used to pass "class" attributes as a workaround

In [31]:
def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                           for attr, value
                           in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                         (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [32]:
tag('br')

'<br />'

In [33]:
tag('p', 'hello')

'<p>hello</p>'

In [34]:
print(tag('p', 'hello', 'world'))

<p>hello</p>
<p>world</p>


In [35]:
tag('p', 'hello', id=33)

'<p id="33">hello</p>'

In [36]:
print(tag('p', 'hello', 'world', cls='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [37]:
tag(content='testing', name="img")

'<img content="testing" />'

In [38]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
          'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

mandatory keyword argument:

In [39]:
def f(a, *, b):
    return a, b

In [40]:
f(1, b=2)

(1, 2)

# Retrieving Information About Parameters

In [41]:
def clip(text, max_len=80):
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:  # no spaces were found
        end = len(text)
    return text[:end].rstrip()

In [42]:
clip.__defaults__

(80,)

In [43]:
clip.__code__

<code object clip at 0x000001A16E0ABBE0, file "<ipython-input-41-17795efb4155>", line 1>

In [44]:
clip.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

In [45]:
clip.__code__.co_argcount

2

## `inspect` module

In [46]:
from inspect import signature

In [47]:
sig = signature(clip)

In [48]:
sig

<Signature (text, max_len=80)>

In [49]:
str(sig)

'(text, max_len=80)'

In [50]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


Binding the functino signature from the tag function to a dict of arguments:

In [51]:
sig = signature(tag)
bound_args = sig.bind(**my_tag)
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>

In [52]:
for name, value in bound_args.arguments.items():
    print(name, '=', value)

name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


In [53]:
del my_tag['name']
bound_args = sig.bind(**my_tag)

TypeError: missing a required argument: 'name'

# Function Annotations

In [54]:
def clip(text:str, max_len:'int > 0'=80) -> str:
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:  # no spaces were found
        end = len(text)
    return text[:end].rstrip()

In [55]:
clip.__annotations__

{'text': str, 'max_len': 'int > 0', 'return': str}

In [56]:
sig = signature(clip)

In [57]:
sig.return_annotation

str

In [58]:
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'> : text = <class 'inspect._empty'>
'int > 0'     : max_len = 80


# Packages for Functional Programming
## The operator module
Factorial:

In [59]:
def fact(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

In [60]:
from operator import mul

In [61]:
def fact(n):
    return reduce(mul, range(1, n+1))

### `itemgetter`
uses `[]` 

Sort a list of tuples by the value of one field:

In [62]:
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),   # <1>
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

In [63]:
from operator import itemgetter

In [64]:
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [65]:
cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


### `attrgetter`
create functinos to extract object attributes by name.

Process a previously defined list of namedtuple

In [66]:
from collections import namedtuple

In [67]:
LatLong = namedtuple('LatLong', 'lat long')

In [68]:
Metropolis = namedtuple('Metropolis', 'name cc pop coord')

In [69]:
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))
               for name, cc, pop, (lat, long) in metro_data]

In [70]:
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [71]:
metro_areas[0].coord.lat

35.689722

In [72]:
from operator import attrgetter

In [73]:
name_lat = attrgetter('name', 'coord.lat')

In [74]:
for city in sorted(metro_areas, key = attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


partial list of functions defined in operator:

In [75]:
import operator

In [76]:
[name for name in dir(operator) if not name.startswith('_')]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

### `methodcaller`
Showing binding of extra arguments:

In [2]:
from operator import methodcaller

In [1]:
s = 'The heart\'s nature is of hollow silence'

In [3]:
upcase = methodcaller('upper')
upcase(s)

"THE HEART'S NATURE IS OF HOLLOW SILENCE"

In [4]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

"The-heart's-nature-is-of-hollow-silence"

In [5]:
str.upper(s)

"THE HEART'S NATURE IS OF HOLLOW SILENCE"

## Freezing Arguments with `functools.partial`

In [82]:
from functools import partial

In [83]:
triple = partial(mul, 3)
triple(7)

21

In [84]:
list(map(triple, range(1, 10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]

Building a convenient Unicode normalizing function with partial

In [85]:
import unicodedata, functools

In [86]:
nfc = functools.partial(unicodedata.normalize, 'NFC')

In [87]:
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [88]:
s1 == s2

False

In [89]:
nfc(s1) == nfc(s2)

True